In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa

In [ ]:
dolar = sc.textFile("DolarVerileri.txt")
def splitTake(line):
    arr = line.split('\t')
    id=int(arr[0])
    date=arr[1]
    dolar=arr[2].replace(".","").replace(",",".")
    return (int(id),date,dolar)
    

dolar = dolar.map(lambda line : splitTake(line)).filter(lambda x: x[2]!="")

dolar = dolar.map(lambda x : (x[0] ,(x[1],float(x[2]))))

dolar.take(5)

[(1, ('02-01-1950', 2.8)),
 (2, ('03-01-1950', 2.8)),
 (3, ('04-01-1950', 2.8)),
 (4, ('05-01-1950', 2.8)),
 (5, ('06-01-1950', 2.8))]

In [ ]:
dolar = sc.textFile("DolarVerileri.txt")


def splitTake(line):
    arr = line.split('\t')
    id=int(arr[0])
    date=arr[1]
    dolar=arr[2].replace(".","").replace(",",".")
    return (int(id),date,dolar)
    
dolar = dolar.map(lambda line : splitTake(line))

dolar_2 = dolar.zipWithIndex()

dolar_2=dolar_2.filter(lambda x: x[0][2]!="")

dolar_2 = dolar_2.map(lambda x : (x[1] ,(x[0][1],float(x[0][2]))))

dolar_2.take(5)


[(0, ('02-01-1950', 2.8)),
 (1, ('03-01-1950', 2.8)),
 (2, ('04-01-1950', 2.8)),
 (3, ('05-01-1950', 2.8)),
 (4, ('06-01-1950', 2.8))]

In [ ]:
joın_dolars=dolar.join(dolar_2)
joın_dolars.take(1)

[(4, (('05-01-1950', 2.8), ('06-01-1950', 2.8)))]

In [ ]:
def fark_alma(data):
  date_1=data[1][0][0]
  dolar_1=data[1][0][1]
  date_2=data[1][1][0]
  dolar_2=data[1][1][1]
  increase=(dolar_2-dolar_1)/dolar_2
  return date_1,dolar_1,date_2,dolar_2,increase


joın_dolars_2=joın_dolars.map(lambda line : fark_alma(line))
joın_dolars_2.map(lambda x: (x[0],x[1],x[2],x[3],x[4])).sortBy(lambda x : x[4],ascending=False).take(10)                

[('19-08-1960', 2.8, '22-08-1960', 9.0, 0.6888888888888889),
 ('24-01-1980', 35.0, '25-01-1980', 70.0, 0.5),
 ('07-08-1970', 9.0, '10-08-1970', 14.85, 0.3939393939393939),
 ('22-02-2001', 685391.0, '23-02-2001', 957879.0, 0.28447016794396784),
 ('05-04-1994', 23031.84, '06-04-1994', 31988.89, 0.2800050267452231),
 ('11-06-1979', 26.5, '12-06-1979', 35.0, 0.24285714285714285),
 ('28-02-1978', 19.25, '01-03-1978', 25.0, 0.23),
 ('06-04-1994', 31988.89, '07-04-1994', 39853.13, 0.1973305484412391),
 ('26-01-1994', 15155.63, '27-01-1994', 17215.5, 0.11965205773866579),
 ('23-02-2001', 957879.0, '26-02-2001', 1072988.0, 0.10727892576617819)]